compress_targets.py


get targets from pkPDB and put them into numpy arrays. This was the code I used to do that.

targets_apdb = [pks, ids]

In [ ]:
import polars as pl
from collections import OrderedDict
import gzip
import time
import subprocess
import numpy as np

to = time.time()

# Read the full CSV, preserving all columns
f="/home/jrhoernschemeyer/Desktop/data_prep/pkas.csv"

df = pl.read_csv(f, separator=";",skip_rows=0)
d=df.to_dict(as_series=False)
a=np.array(list(d.values()))
sorted_indices = np.argsort(a)

# Squeeze to remove the extra dimension
a_flat = a.squeeze()               # Now shape becomes (12628148,)
indices_flat = sorted_indices.squeeze()

# Use the flattened indices to retrieve sorted data
sorted_data = a_flat[indices_flat]

parts = np.char.partition(sorted_data, ";")

pdbs=parts[:,0]
resi_data=parts[:,2]
arr = pdbs#np.array([...])  # your sorted data

unique_vals, counts = np.unique(arr, return_counts=True)

badtargets=[]

print("unique pdbs gotten, starting parsing targets", (to - time.time())/60)
code = {"H":"0",
        "A":"1",
        "L":"2",
        "T":"3",
        "G":"4",
        "C":"5"}
refpk = {
    "A": np.float32(3.79),
    "C": np.float32(8.67),
    "G": np.float32(4.20),
    "H": np.float32(6.74),
    "L": np.float32(10.46),
    "T": np.float32(9.59),
}

#
split_indices = np.cumsum(counts)[:-1]
subs=np.split(resi_data,split_indices)

for pdbdata,pdb in zip(subs,unique_vals):
    try:
        par = np.char.rpartition(pdbdata, ";")
        info = [np.char.split(np.atleast_1d(line[0]),";")[0] for line in par if line[0]]
        
        ids= np.char.encode([np.char.add(np.char.add(i[2],i[0]),code[i[1][0]]) for i in info if not np.char.startswith(i[1],"CT") and not np.char.startswith(i[1],"N") ])
        #ids = [np.char.add(np.char.add(i[2],i[0]),code[i[1][0]]) for i in info if not np.char.startswith(i[1],"CT") and not np.char.startswith(i[1],"N") ]
        
        pks=[np.round(np.float32(p[2]) - refpk[i[1][0][0]],6) for i,p in zip(info,par) if not np.char.startswith(i[1],"CT") and not np.char.startswith(i[1],"N")]

        np.savez_compressed(f"/home/jrhoernschemeyer/Desktop/data_prep/targets/{pdb}.npz",pks=pks,ids=ids)
        #np.load(f"/home/jrhoernschemeyer/Desktop/data_prep/targets/{pdb}.npz")
    except Exception as e:
        badtargets.append(pdb)
        print(pdb)
        print(e)

with gzip.open("/home/jrhoernschemeyer/Desktop/data_prep/badtargets.gz", "wb") as f:
    for pdb in badtargets:
        f.write(pdb.encode())
        f.write(b" ")



print("finished", (time.time()-to)/60)

generate inputs with the "compressed" target files and structure data.

from each line, get the coors, atomic number, chain, res number, and resname if the line is a titratable residue.

then, group them by resnumber.

generate_inputs.py

In [ ]:
import glob
import gzip
import os
from itertools import groupby
import numpy as np

elements = {
    b"D":1, b"H": 1, "LI":3, b"C": 6, b"N": 7, b"O": 8, 
    b"F":9,
    b"NA": 11, b"MG": 12,  b"P": 15, b"S": 16, b"CL": 17, b"K": 19,
    b"CA": 20, b"MN": 25, b"FE": 26, b"CO":27, b"NI":28,
    b"CU": 29, b"ZN": 30,
    b"SE":34,
    b"MO":42, b"SN":50, 
    b"I":53, b"CS":55,
    b"W":74, b"PT":78
    }
code={b"HIS":b"0",
        b"ASP":b"1",
        b"LYS":b"2",
        b"TYR":b"3",
        b"GLU":b"4",
        b"CYS":b"5"}

target_files=glob.glob("/home/jrhoernschemeyer/Desktop/data_prep/targets/*.npz")

def is_float(bs):
    if b'.' not in bs:
        return False
    else:
        return bs


def parsepdb(file):
    pdblines=[]
 
    with gzip.open(file, "r") as f:
        l=f.readlines()
        lines=np.char.array(f.readlines())
        f.close()

    lines=[line for line in l if np.char.startswith(line,b"ATOM")]
    

    for i,l in enumerate(lines):
        l=l.split()

        #resname 
        info=l[3] 
        if info in code.keys(): #titratable
            aline=[code[info]]
        else:
            continue
        
        #atomnumber
        try:
            info=elements[l[-1].strip(b"+-0123456789")]
        except:
            try:
                info=elements[l[2]]
            except:
                continue
        aline.append(info)

        #coors
        xyz=[]
        info=(l[6],l[7],l[8])
        for c in info:
            if is_float(c):
                xyz.append(c)
        
        if len(xyz) < 3:
            continue
        else:
            aline.append(xyz[0])
            aline.append(xyz[1])
            aline.append(xyz[2])

        #chain   
        chain=l[4]
        if is_float(chain):
            continue
        else:
            aline.append(chain) 

        #resnum
        resnum=l[5]
        if is_float(resnum):
            continue
        else:
            aline.append(resnum.strip(b"ABCDEFGHIJKLMNOPQRSTUVWXYZ")) 


        pdblines.append(aline)
        
    #whole pdb
    pdblines = [list(group) for key, group in groupby(pdblines, key=lambda row: row[-1])]
    
    #myinfo
    zs=np.array([[np.array([np.uint8(l[1]) for l in resi])] for resi in pdblines])
    coors=np.array([np.array([[np.float32(l[2]),np.float32(l[3]),np.float32(l[4])] for l in resi]) for resi in pdblines])
    pdblines=[r[0] for r in pdblines]
    ids=np.char.array([np.char.add(np.char.add(l[-1],l[-2]),l[0]) for l in pdblines])

    #return
    if pdblines:
        return ids, zs, coors
    else:
        return None, None, None
    

    #else:
        #continue

failparse=[]
for file in target_files:
    pdb=file[-8:-4]
    array=np.load(file)
    pks,pids = array["pks"], np.char.array(array["ids"])
    
    path=f"/home/jrhoernschemeyer/Desktop/data_prep/structures/pdbs/{pdb}.pdb.gz"
    if os.path.exists(path):
        mids, zs, coors =parsepdb(path)
        if type(mids) != None:
            common_vals, pidx, midx = np.intersect1d(pids, mids,return_indices=True)
            ids=pids[pidx]
            pks=pks[pidx]
            pos=coors[midx]
            species=zs[midx]
            np.savez_compressed(f"/home/jrhoernschemeyer/Desktop/data_prep/inputs/{pdb}.npz",z=species, pos=pos, pks=pks,ids=ids)
            break
        else:
            failparse.append(pdb)

with gzip.open("/home/jrhoernschemeyer/Desktop/data_prep/failparse_final.gz","wb") as f:
    for pdb in failparse:
        f.write(pdb.encode())
        f.write(" ")
        

/home/jrhoernschemeyer/.local/lib/python3.6/site-packages/ipykernel_launcher.py:126: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/jrhoernschemeyer/.local/lib/python3.6/site-packages/ipykernel_launcher.py:127: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


This is how theyll be loaded into a model.

In [264]:
a=np.load(f"/home/jrhoernschemeyer/Desktop/data_prep/inputs/{pdb}.npz",allow_pickle=True)
import torch 
for b in a["pos"]:
    print(torch.tensor(list(b)))

tensor([[ 7.2810, 20.6940, -2.2570],
        [ 8.0150, 19.4630, -2.5260],
        [ 9.4160, 19.4750, -1.8930],
        [ 9.9050, 18.4360, -1.4360],
        [ 8.1240, 19.2350, -4.0360],
        [ 8.6280, 17.8530, -4.4240],
        [ 7.5550, 16.8050, -4.1470],
        [ 8.0330, 15.4070, -4.4810],
        [ 6.9940, 14.3960, -4.1380]])
tensor([[10.0600, 20.6450, -1.8670],
        [11.3930, 20.7500, -1.2690],
        [11.3030, 20.5710,  0.2430],
        [12.2570, 20.1270,  0.8730],
        [12.0480, 22.1040, -1.5810],
        [12.4630, 22.3200, -3.0280],
        [12.0490, 21.4600, -4.0340],
        [13.2900, 23.3870, -3.3780],
        [12.4480, 21.6540, -5.3520],
        [13.6900, 23.5860, -4.6760],
        [13.2690, 22.7190, -5.6610],
        [13.6800, 22.9090, -6.9640]])
tensor([[ 9.2750, 18.5920,  1.5920],
        [ 9.1620, 17.1490,  1.7330],
        [10.5860, 16.5460,  1.6870],
        [10.9350, 15.6730,  2.4920],
        [ 8.2980, 16.5450,  0.6160],
        [ 8.0440, 15.0570,  0.8170],

'/home/jrhoernschemeyer/Desktop/data_prep/structures/pdbs/6fwu.pdb.gz' Hs